# Project 1 - TripAdvisor reccomandation system

# DATA PREPARATION

## Load the dataset

### Download dataset

In [2]:
import kagglehub
import zipfile
import os

dataset_filename = 'reviews.csv'
cache_dataset_zipped = kagglehub.dataset_download("joebeachcapital/hotel-reviews", path=dataset_filename)
# the file is actually downloaded as zipped, even if there is not '.zip' extensions
# so we need to extract it
extract_path = './'
# Check if the file already exists
if not os.path.exists(extract_path + dataset_filename):
    with zipfile.ZipFile(cache_dataset_zipped, 'r') as zip_ref:
        # Iterate through each file in the zip file
        for file in zip_ref.namelist():
            # Check if the file already exists
            if not os.path.exists(os.path.join(extract_path, file)):
                # Extract the file
                zip_ref.extract(file, extract_path)
        
dataset_path = extract_path + dataset_filename

### import dataset

In [3]:
import pandas as pd

df = pd.read_csv(dataset_path)
df.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


## Data formatting

### Flatten the json 'ratings' column

In [4]:
from pandas import json_normalize
# convert JSON column to DataFrame
json_df = json_normalize(df.ratings.apply(eval))

# merge the two dataframes
df = df.drop(columns=['ratings']).join(json_df)
df

,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,service,cleanliness,overall,value,location,sleep_quality,rooms,check_in_front_desk,business_service_(e_g_internet_access)
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
2,“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False,4.0,5.0,4.0,4.0,5.0,4.0,4.0,NaN,NaN
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False,5.0,5.0,4.0,5.0,5.0,5.0,5.0,NaN,NaN
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False,4.0,5.0,4.0,3.0,5.0,5.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878556,“vil komme igen”,"Pænt hotel i et livligt miljø. Store værelser,...","{'username': '', 'id': '', 'location': ''}",NaN,84093,0,2008-08-31,51441576,False,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
878557,“excellent”,"un excellent hotel 4 *, pour un prix raisonnab...","{'username': 'dan016', 'num_reviews': 2, 'num_...",July 2008,84093,0,2008-07-18,18003332,False,4.0,5.0,4.0,4.0,4.0,NaN,5.0,5.0,NaN
878558,“Un hotel eccezionale”,"L'hotel, situato in una zona tranquilla e a du...","{'username': '', 'id': '', 'location': ''}",July 2008,84093,0,2008-04-18,15564515,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN
878559,“Gerne wieder”,Sehr schön ausgestattetes Hotel in bester Lage...,"{'username': '', 'id': '', 'location': ''}",July 2008,84093,0,2008-04-01,15564508,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN


### Remove unused columns and rows

In [5]:
# unused columns
df.drop(columns=['title', 'author', 'date_stayed', 'num_helpful_votes', 'date', 'id', 'via_mobile'], inplace=True, errors='ignore')
# reviews that have null values on relevant rating criteria
relevant_ratings = ['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']
df.dropna(subset=relevant_ratings, inplace=True)
# reviews that have more rating criteria than the relevant ones
df = df[~df['check_in_front_desk'].notna()]
df = df[~df['business_service_(e_g_internet_access)'].notna()]
df.drop(columns=['check_in_front_desk', 'business_service_(e_g_internet_access)'], inplace=True, errors='ignore')
df

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
0,Stayed in a king suite for 11 nights and yes i...,93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,"On every visit to NYC, the Hotel Beacon is the...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,This is a great property in Midtown. We two di...,1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0
3,The Andaz is a nice hotel in a central locatio...,1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0
4,I have stayed at each of the US Andaz properti...,1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...
878548,"Das Hotel Palomar gehoert zur Kimpton Gruppe, ...",84093,4.0,5.0,4.0,3.0,4.0,4.0,4.0
878549,Anlässlich einer Geschäftsreise waren wir das ...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0
878550,"Das Hotel liegt in Arlington, eine etwas gehob...",84093,5.0,5.0,5.0,4.0,5.0,5.0,5.0
878551,Mon mari et moi sommes restés à l'hôtel du jeu...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0


# DATA PROCESSING
language detection, tokenization, lowercasing, removing puntualization, speling correction, stopwords...

## Languages detection

### Inspect

In [6]:
# ensures no empty strings
print('empty reviews count:', df.text.apply(lambda x: x.strip() == '').sum())
df.isnull().sum()

empty reviews count: 0


text             0
offering_id      0
service          0
cleanliness      0
overall          0
value            0
location         0
sleep_quality    0
rooms            0
dtype: int64

In [7]:
# assess the reviews are mostly long, therefore helping in language detection
short_len_def = 70
short_reviews_percentage = round((len(df[df['text'].str.len() < short_len_def]) / len(df)) * 100, 2)
print(f'short (less than {short_len_def} character long) reviews: {short_reviews_percentage}%')

short reviews: 0.31%


### Detect

In [8]:
# Google library to detect the language of a review
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# ensure consistent results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

if 'language' not in df.columns:
    df_temp = df.copy()
    df['language'] = df_temp['text'].apply(detect_language)

df

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms,language
0,Stayed in a king suite for 11 nights and yes i...,93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0,en
1,"On every visit to NYC, the Hotel Beacon is the...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0,en
2,This is a great property in Midtown. We two di...,1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0,en
3,The Andaz is a nice hotel in a central locatio...,1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0,en
4,I have stayed at each of the US Andaz properti...,1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0,en
...,...,...,...,...,...,...,...,...,...,...
878548,"Das Hotel Palomar gehoert zur Kimpton Gruppe, ...",84093,4.0,5.0,4.0,3.0,4.0,4.0,4.0,de
878549,Anlässlich einer Geschäftsreise waren wir das ...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0,de
878550,"Das Hotel liegt in Arlington, eine etwas gehob...",84093,5.0,5.0,5.0,4.0,5.0,5.0,5.0,de
878551,Mon mari et moi sommes restés à l'hôtel du jeu...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0,fr


In [15]:
# Calculate the percentage of each language
language_counts = df['language'].value_counts(normalize=True) * 100
language_counts.rename('percentage', inplace=True)

KeyError: 'language'

In [10]:
# visual quality check (second most common language)
df[df['language'] == language_counts[1]].head()

/var/folders/yg/gp8smp8d1cx2j1t_sxxl13580000gn/T/ipykernel_88935/3143627420.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[df['language'] == language_counts[1]].head()


,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms,language


### Filter
Since the dataset is mostly in English, we can filter out the non-English reviews.

In [11]:
# keep only english reviews
df = df[df['language'] == 'en']
df.drop(columns=['language'], inplace=True)

/var/folders/yg/gp8smp8d1cx2j1t_sxxl13580000gn/T/ipykernel_88935/1681499117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['language'], inplace=True)


### Different approach

In [12]:
# Approccio diverso per rilevarer la lingua, ma non è molto efficiente
# per parole
to_filter = ['bonne','de','le','les','une','des','du','ce','cet','cette','ces','mon','ma','mes','ton','ta','tes','sa','ses','notre','nos','votre','vos','leur','leurs','quel','quelle','quels','quelles','ceci','cela','celui','celle','ceux','celles']
pattern = r'\b(?:' + '|'.join(to_filter) + r')\b'
# per caratteri
to_filter = ['â','ä','ç','é','è','ê','ë','î','ï','ô','ö','ù','û','ü','ÿ']
pattern = '|'.join(to_filter)
# Filter the DataFrame
df_filtered = df[df['text'].str.contains(pattern, case=False, na=False, regex=True)]

# Display the resulting DataFrame
df_filtered

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
44,The information about the Sherry-Netherland is...,93559,2.0,3.0,2.0,1.0,5.0,3.0,3.0
77,My wife and I stayed in four hotels in our rec...,1456560,4.0,5.0,5.0,4.0,5.0,5.0,5.0
173,I had booked my stay at the Setai as a 25th we...,1776857,1.0,4.0,2.0,2.0,4.0,3.0,5.0
265,The Sherry-Netherland Hotel was a perfect spot...,93559,5.0,5.0,5.0,4.0,5.0,5.0,5.0
456,Just got back from a 3 night stay at the Beaco...,93338,3.0,5.0,5.0,4.0,4.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...
873896,I recently spent a weekend with my girlfriend ...,258705,5.0,5.0,5.0,5.0,5.0,5.0,5.0
874168,"My fiancé and I, stayed at the Palomar for 3 n...",84093,1.0,5.0,3.0,5.0,5.0,5.0,5.0
877227,I just returned from a two-night stay at the M...,83040,5.0,5.0,4.0,3.0,5.0,3.0,4.0
877677,We liked everything about the Hotel Palomar: t...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0


## Tokenization

In [13]:
import nltk
nltk.download('punkt_tab')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt_tab to /Users/Marco/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### By sentence

In [14]:
df['text'] = df['text'].apply(lambda x: tokenizer.tokenize(x))
df.head()

/var/folders/yg/gp8smp8d1cx2j1t_sxxl13580000gn/T/ipykernel_88935/4286509684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: tokenizer.tokenize(x))


,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
0,[Stayed in a king suite for 11 nights and yes ...,93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,"[On every visit to NYC, the Hotel Beacon is th...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,"[This is a great property in Midtown., We two ...",1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0
3,[The Andaz is a nice hotel in a central locati...,1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0
4,[I have stayed at each of the US Andaz propert...,1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0


## Removing punctuation

### By word
Ha senso fare prima per frasi e poi per parole?? Ce l'ha fatto fare nel TD

In [21]:
from nltk.tokenize import word_tokenize
df['text'] = df['text'].apply(lambda x: [nltk.word_tokenize(sentence) for sentence in x])
df.head()

/var/folders/yg/gp8smp8d1cx2j1t_sxxl13580000gn/T/ipykernel_88935/2170852960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: [nltk.word_tokenize(sentence) for sentence in x])


,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
0,"[[Stayed, in, a, king, suite, for, 11, nights,...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,"[[On, every, visit, to, NYC, ,, the, Hotel, Be...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,"[[This, is, a, great, property, in, Midtown, ....",1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0
3,"[[The, Andaz, is, a, nice, hotel, in, a, centr...",1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0
4,"[[I, have, stayed, at, each, of, the, US, Anda...",1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0


## Lowercasing

## Spelling correction

## Stopwords removal

## Stemming

## Concatenate reviews

In [70]:
df = df.groupby('offering_id').agg({
    'text': ' '.join, 
    'service': 'mean',
    'cleanliness': 'mean',
    'overall': 'mean',
    'value': 'mean',
    'location': 'mean',
    'sleep_quality': 'mean',
    'rooms': 'mean'
}).reset_index()

# Display the resulting DataFrame
df

Before concatenation: 3754


,offering_id,text,service,cleanliness,overall,value,location,sleep_quality,rooms
0,72572,I had to make fast visit to seattle and I foun...,4.601010,4.636364,4.388889,4.323232,4.570707,4.333333,4.282828
1,72579,"Great service, rooms were clean, could use som...",4.232000,4.240000,3.888000,4.152000,4.192000,3.768000,3.856000
2,72586,Beautiful views of the space needle - especial...,4.250000,4.287879,4.045455,4.053030,4.537879,4.113636,3.992424
3,72598,This hotel is in need of some serious updates....,3.243243,3.243243,2.918919,3.054054,3.027027,3.270270,3.189189
4,73236,My experience at this days inn was perfect. th...,4.277778,3.111111,3.388889,3.777778,4.111111,3.722222,3.222222
...,...,...,...,...,...,...,...,...,...
3749,3523356,I've stayed at plenty of Hampton Inns during m...,4.928571,4.928571,4.571429,4.214286,4.500000,4.571429,4.500000
3750,3541823,"Inn staff absolutely wonderful, helpful, knowl...",4.333333,4.500000,3.666667,4.000000,4.666667,3.166667,3.000000
3751,3572384,"Crowded, noisy, dirty. Service is poor, food i...",3.000000,2.000000,2.000000,2.000000,4.000000,3.000000,2.000000
3752,3572583,El hotel estaba en medio de una remodelación. ...,1.000000,1.000000,1.000000,1.000000,4.000000,3.000000,1.000000


# TRAINING

### BM25 documentation example

In [9]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!", # documento 1
    "It is quite windy in London", # documento 2
    "How is the weather today?" # documento 3
]

# in input ci deve essere una lista di liste di stringhe
model_input = [doc.split(" ") for doc in corpus]
print(model_input)
tokenized_corpus = model_input

bm25 = BM25Okapi(tokenized_corpus)

[['Hello', 'there', 'good', 'man!'], ['It', 'is', 'quite', 'windy', 'in', 'London'], ['How', 'is', 'the', 'weather', 'today?']]


In [10]:
# prova
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([0.        , 0.93729472, 0.        ])

In [11]:
# per recuperare i best n documenti
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

# VALIDATION